# The Julia Opportunistic Network Simulator

In [1]:
#using Pkg
#Pkg.instantiate()

push!(LOAD_PATH, "../src")
push!(LOAD_PATH, "src")

using Random
using DataFrames
using JONS


## Configuration

In [2]:
MOVEMENT_FILE="data/large_1.one"
SCAN_INTERVAL=2.0
NET_RANGE=100
ANIMATION_STEPS = 20.0
VIZ = false
POSLOG = false

false

In [3]:
one_scenario = parse_one_movement(MOVEMENT_FILE)

OneScenario(duration=3600.0, nn=17, w=1000.0, h=1000.0, #movements=18012)

In [4]:
Random.seed!(1)
#plotlyjs()

# configure network
network = NetworkSettings(NET_RANGE, 54000000)

# configure nodes
epidemic = EpidemicRouter(10000, SCAN_INTERVAL)
saw = SprayAndWaitRouter(10000, SCAN_INTERVAL, 7)
nodes = generate_nodes(one_scenario.nn, network, epidemic)

msggenconfig = MessageGeneratorConfig("M", (80, 120), (20.0, 20.0), (6, 15), (17, 17), Burst)
config = Dict()
config["visualize"] = VIZ
config["poslogger"] = POSLOG
#sim = NetSim(last_move_time + 1, (Float32(WORLD_SIZE[1]), Float32(WORLD_SIZE[2])), nodes, movements, MessageGeneratorConfig[msggenconfig], config)
sim = NetSim(one_scenario.duration + 1, (one_scenario.w, one_scenario.h), nodes, one_scenario.movements, MessageGeneratorConfig[msggenconfig], config)

sim_init(sim)
sim_run(sim)

┌ Info: initializing simulation
└ @ JONS /Users/lab/src/scratch/JONS.jl/src/simulation.jl:5
┌ Info: running simulation
└ @ JONS /Users/lab/src/scratch/JONS.jl/src/simulation.jl:45


┌ Info: real: 2623 milliseconds sim: 3602 rate: 1373.24 s/s
└ @ JONS /Users/lab/src/scratch/JONS.jl/src/simulation.jl:75
┌ Info: simulation finished in 2 seconds, 623 milliseconds
└ @ JONS /Users/lab/src/scratch/JONS.jl/src/simulation.jl:76


## Results

In [5]:
DataFrame(net_stats(sim))

Row,drop,rx,tx
,Int64,Int64,Int64
1,0,104363,104363


In [6]:
DataFrame(bundle_stats(sim))

Row,aborted,created,delivered,delivery_prob,dropped,dups,hops_avg,latency_avg,overhead_ratio,relayed,removed,started
,Int64,Int64,Int64,Float64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64
1,0,1800,1664,0.924444,0,81287,3.07151,241.187,61.7181,104363,0,105808


In [7]:
sim_viz(sim)

## Batch Simulation

In [8]:
const NUM_RUNS = 3
const NUM_NODES = 10
const DURATION = 3600.0
const WORLD_SIZE = (1000, 1000)

#plotlyjs()

# configure network
network = NetworkSettings(NET_RANGE, 54000000)

# configure nodes
routers = [EpidemicRouter(10000, SCAN_INTERVAL), 
  SprayAndWaitRouter(10000, SCAN_INTERVAL, 7), 
  SprayAndWaitRouter(10000, SCAN_INTERVAL, 2),
  SprayAndWaitRouter(10000, SCAN_INTERVAL, 1)]
#run_idx = 1

dfs = []

@time for run_idx in 1:NUM_RUNS
  Random.seed!(run_idx)
  one_scenario = generate_randomwaypoint_movement(DURATION, NUM_NODES, Float32(WORLD_SIZE[1]), Float32(WORLD_SIZE[2]), Float32(1.0), Float32(5.0), Float32(0.0), Float32(60.0))


  for r in routers
    println("#run ", run_idx, " w/ router: ", r)
    r_strategy = string(r)
    if r_strategy == "SprayAndWait"
      r_strategy = "SprayAndWait_" * string(r.core.config["copies"])
    end
    #run_idx += 1

    nodes = generate_nodes(one_scenario.nn, network, r)
    
    msggenconfig = MessageGeneratorConfig("M", (80, 120), (20.0, 20.0), (1, one_scenario.nn), (1, one_scenario.nn), Single)
    config = Dict()
    config["visualize"] = VIZ
    config["poslogger"] = POSLOG
    #sim = NetSim(last_move_time + 1, (Float32(WORLD_SIZE[1]), Float32(WORLD_SIZE[2])), nodes, movements, MessageGeneratorConfig[msggenconfig], config)
    sim = NetSim(one_scenario.duration + 1, (one_scenario.w, one_scenario.h), nodes, one_scenario.movements, MessageGeneratorConfig[msggenconfig], config)
    
    sim_init(sim)
    sim_run(sim)

    run_df = DataFrame(bundle_stats(sim))
    colname = "Strategy"
    #run_df[!, colname] = string(r)
    insertcols!(run_df, 1, colname => r_strategy)
    colname = "run"
    insertcols!(run_df, 2, colname => run_idx)    
    push!(dfs, run_df)
  end
end

df = popfirst!(dfs)
for d in dfs
  df = vcat(df, d)
end
df

#run 1 w/ router: Epidemic


┌ Info: initializing simulation
└ @ JONS /Users/lab/src/scratch/JONS.jl/src/simulation.jl:5
┌ Info: running simulation
└ @ JONS /Users/lab/src/scratch/JONS.jl/src/simulation.jl:45


BoundsError: BoundsError: attempt to access 10-element Vector{Node} at index [6:15]

In [9]:
using Plots
using Statistics

g_strat = groupby(df, :Strategy)
df1 = combine(g_strat, [:delivery_prob] .=> mean)
p1 = bar(df1[!,"Strategy"], df1[!, "delivery_prob_mean"], label="")
title!("Delivery Probability (mean)")

df1 = combine(g_strat, [:hops_avg] .=> mean)
p2 = bar(df1[!,"Strategy"], df1[!, "hops_avg_mean"], label="")
title!("Hop Count (mean)")

df1 = combine(g_strat, [:latency_avg] .=> mean)
p3 = bar(df1[!,"Strategy"], df1[!, "latency_avg_mean"], label="")
title!("Latency (mean)")
ylabel!("seconds")

df1 = combine(g_strat, [:overhead_ratio] .=> mean)
p4 = bar(df1[!,"Strategy"], df1[!, "overhead_ratio_mean"], label="")
title!("Overhead Ratio (mean)")

plot(p1,p2, p3, p4, layout=(2,2), size=(1000,1000), plot_title="Results over " * string(NUM_RUNS) * " runs")



UndefVarError: UndefVarError: df not defined